In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from mlcv import generate_points, render_pcl

# fix a random seed for numpy so we get the same point cloud in every run
np.random.seed(1)
# generate a set of data points 
data,ground_truth = generate_points(noise=[0.01]*3, distribution=[200 ]*3)
# render the resulting point cloud
fig = plt.figure(figsize=(4,4))
ax1 = fig.add_subplot(1,1,1,projection='3d')
render_pcl(data,ground_truth,ax=ax1)

In [ ]:
from mlcv import distance_to_origin, cost, reduced_cost, greedy_search, render_stats
from tqdm.notebook import trange, tqdm

# define cost functions
def cf_prime(data,combs):
    dthr = 0.1
    return distance_to_origin(data,combs) - dthr

def cf(data,combs):
    return np.zeros(combs.shape[0])

# initialize indexing
indexing = np.random.choice(1,data.shape[0])
# initialize stopping criteria, set up list for stats gathered over runtime
stop, stats = 2000, []
# initialize local search algorithm
alg = greedy_search(data, indexing, cf, cf_prime, stop=stop, N=50, M=4000)

# create iterator over resulting indexings with nice looking progress-bar
current_indexing = indexing
pbar = tqdm(alg, total=stop)
for indexing, v, k, stats_bm in pbar:
    current_indexing = indexing
    stats.append(stats_bm)
    pbar.set_description("partitions %03d" % stats_bm["partcount"])

# render statistics
render_stats(stats)
plt.tight_layout()

In [ ]:
from mlcv import render_overlap
# render guessed partitions & compare with ground truth information
fig = plt.figure(figsize=(8,6))
ax1 = fig.add_subplot(1,2,1,projection='3d')
ax2 = fig.add_subplot(1,2,2)
render_pcl(data, current_indexing, ax=ax1)
render_overlap(ground_truth, current_indexing, ax=ax2, 
               indexing1name="ground truth", indexing2name="predicted")